In [41]:
# Load modules

import numpy as np
import matplotlib.pyplot as plt
import datetime

In [24]:
# Define functions

def cartesian_to_spherical(coords):   
    r = np.sqrt(coords[:,0]**2 + coords[:,1]**2 + coords[:,2]**2)
    phi = np.sign(coords[:,0]) * np.arccos(coords[:,1] / (np.sqrt(coords[:,0]**2 + coords[:,1]**2)))
    # theta = np.nan_to_num(x=theta)    - can use this for nan error if necessary
    theta = np.arccos(coords[:,2]/r)
    return np.array([r, theta, phi]).transpose()

def data_processing(data_path):
    # juice to callisto
    juice_wrt_callisto = np.loadtxt(data_path, delimiter=",", unpack=True)
    juice_wrt_callisto = juice_wrt_callisto.transpose()
    Xjc, Yjc, Zjc, vxjc, vyjc, vzjc, t = juice_wrt_callisto

    # convert positions to m
    juice_wrt_callisto_cart = 1e3 * np.array([Xjc, Yjc, Zjc]).transpose()

    juice_wrt_callisto_spher = cartesian_to_spherical(juice_wrt_callisto_cart)

    return [t, juice_wrt_callisto_cart, juice_wrt_callisto_spher]

def CA_info(orbit):
    min_index = np.argmin(orbit[4])
    return orbit[min_index]

In [25]:
### Load in data from SPICE kernels, data loaded in has units km and km/s
# data paths
data_paths = []
data_path = "./spice_data/juice_wrt_callisto_C"

for i in range(1, 22):
    data_paths.append(data_path + str(i) + ".csv")

print(data_paths)

['./spice_data/juice_wrt_callisto_C1.csv', './spice_data/juice_wrt_callisto_C2.csv', './spice_data/juice_wrt_callisto_C3.csv', './spice_data/juice_wrt_callisto_C4.csv', './spice_data/juice_wrt_callisto_C5.csv', './spice_data/juice_wrt_callisto_C6.csv', './spice_data/juice_wrt_callisto_C7.csv', './spice_data/juice_wrt_callisto_C8.csv', './spice_data/juice_wrt_callisto_C9.csv', './spice_data/juice_wrt_callisto_C10.csv', './spice_data/juice_wrt_callisto_C11.csv', './spice_data/juice_wrt_callisto_C12.csv', './spice_data/juice_wrt_callisto_C13.csv', './spice_data/juice_wrt_callisto_C14.csv', './spice_data/juice_wrt_callisto_C15.csv', './spice_data/juice_wrt_callisto_C16.csv', './spice_data/juice_wrt_callisto_C17.csv', './spice_data/juice_wrt_callisto_C18.csv', './spice_data/juice_wrt_callisto_C19.csv', './spice_data/juice_wrt_callisto_C20.csv', './spice_data/juice_wrt_callisto_C21.csv']


In [26]:
orbit1 = data_processing('./spice_data/juice_wrt_callisto_C1.csv')
#orbit1 =  np.asarray(orbit1)
print(orbit1)

[array([1.02465727e+09, 1.02465733e+09, 1.02465739e+09, 1.02465745e+09,
       1.02465751e+09, 1.02465757e+09, 1.02465763e+09, 1.02465769e+09,
       1.02465775e+09, 1.02465781e+09, 1.02465787e+09, 1.02465793e+09,
       1.02465799e+09, 1.02465805e+09, 1.02465811e+09, 1.02465817e+09,
       1.02465823e+09, 1.02465829e+09, 1.02465835e+09, 1.02465841e+09,
       1.02465847e+09, 1.02465853e+09, 1.02465859e+09, 1.02465865e+09,
       1.02465871e+09, 1.02465877e+09, 1.02465883e+09, 1.02465889e+09,
       1.02465895e+09, 1.02465901e+09, 1.02465907e+09, 1.02465913e+09,
       1.02465919e+09, 1.02465925e+09, 1.02465931e+09, 1.02465937e+09,
       1.02465943e+09, 1.02465949e+09, 1.02465955e+09, 1.02465961e+09,
       1.02465967e+09, 1.02465973e+09, 1.02465979e+09, 1.02465985e+09,
       1.02465991e+09, 1.02465997e+09, 1.02466003e+09, 1.02466009e+09,
       1.02466015e+09, 1.02466021e+09, 1.02466027e+09, 1.02466033e+09,
       1.02466039e+09, 1.02466045e+09, 1.02466051e+09, 1.02466057e+09,
     

In [33]:
orbits_all = {}

for i in range(len(data_paths)):
    t, cart, spher = data_processing(data_paths[i])
    z = np.c_[t, cart]
    z = np.c_[z, spher]
    orbits_all['orbit%s' % (i+1)] = z

[[ 1.02465727e+09  1.45193206e+07  2.72363878e+07 ...  3.08823793e+07
   1.53698531e+00  4.89764277e-01]
 [ 1.02465733e+09  1.44374223e+07  2.69385402e+07 ...  3.05814244e+07
   1.53650318e+00  4.91983994e-01]
 [ 1.02465739e+09  1.43553545e+07  2.66407169e+07 ...  3.02805712e+07
   1.53601162e+00  4.94242588e-01]
 ...
 [ 1.02466885e+09 -6.72085199e+06 -2.90719574e+07 ...  2.98674503e+07
   1.52692275e+00 -2.91440394e+00]
 [ 1.02466891e+09 -6.85654365e+06 -2.93496053e+07 ...  3.01682357e+07
   1.52742512e+00 -2.91209246e+00]
 [ 1.02466897e+09 -6.99237476e+06 -2.96271602e+07 ...  3.04691278e+07
   1.52791777e+00 -2.90982157e+00]]


In [47]:
i = 1
CA_times_utc = []
CA_times_utc_min12hr = []
for orbit, vector in orbits_all.items():
    CA_info_vector = CA_info(vector)
    globals()['CA_orbit%s' % (i)] = CA_info_vector
    CA_time_utc = datetime.datetime(2000, 1, 1, 0, 0, 0) + datetime.timedelta(seconds=CA_info_vector[0])
    CA_time_utc_min12hr = CA_time_utc + datetime.timedelta(hours=-12)
    CA_times_utc.append(CA_time_utc)
    CA_times_utc_min12hr.append(CA_time_utc_min12hr)

    i += 1

for times in CA_times_utc_min12hr:
    print(times)

2032-06-19 23:07:09.184390
2032-07-28 00:33:09.183340
2032-08-13 16:48:09.182950
2032-09-09 18:04:09.182490
2032-09-26 10:34:09.182360
2032-10-13 03:04:09.182360
2032-10-29 19:34:09.182500
2032-11-15 11:49:09.182770
2032-12-02 04:19:09.183140
2033-02-23 14:49:09.185310
2033-03-12 07:04:09.185540
2033-05-09 14:48:09.185340
2033-06-03 17:19:09.184820
2033-06-20 09:34:09.184380
2033-07-07 02:04:09.183920
2033-07-23 18:19:09.183460
2033-08-09 10:49:09.183050
2033-10-31 21:34:09.182530
2034-02-13 21:18:09.185110
2034-04-30 19:37:09.185470
2034-06-23 00:49:09.184320


2032-06-20 11:07:09.184390
2032-06-19 23:07:09.184390
